In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
captureSize = 35.8 #for tmb claculation
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)

In [2]:
tcga_data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/'
! ls -lh /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/*.TABLE

-rw-rw-r-- 1 was966 zitnik 4.5M Nov 16 18:27 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/GENE.TABLE
-rw-rw-r-- 1 was966 zitnik 2.1M Nov 16 18:25 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/MYGENE.TABLE
-rw-rw-r-- 1 was966 zitnik  75K Oct 12 15:25 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/NORMAL.PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 314M Oct 12 15:25 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/NORMAL.TPM.TABLE
-rw-rw-r-- 1 was966 zitnik 1.1M Nov 16 18:28 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/PATIENTS.TABLE
-rw-rw-r-- 1 was966 zitnik 1.6M Nov 10 15:44 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/PATIENTS.v2.TABLE
-rw-rw-r-- 1 was966 zitnik 4.4G Nov 16 18:28 /n/data1/hms/dbmi/zitnik/lab/users/was966/TCGA/GDC_v37/TPM.TABLE


In [3]:

## TCGA dataset
tcga_gene = pd.read_pickle(os.path.join(tcga_data_path, 'GENE.TABLE'))
tcga_patient = pd.read_pickle(os.path.join(tcga_data_path, 'PATIENTS.TABLE'))
tcga_tpm = pd.read_pickle(os.path.join(tcga_data_path, 'TPM.TABLE'))


In [4]:
dfm = pd.read_excel(os.path.join(tcga_data_path, 'Noushin_Nat_Med_2023.xlsx'), sheet_name='ST3', skiprows=[0], index_col=0)
describe = dfm.iloc[-1]
dfm = dfm.iloc[:-1]

In [5]:
len(dfm)

9256

In [6]:
dfm1 = dfm[['Stage', 'OS (Months)', 'OS Cens. (1=Dead,0=Alive)', 'TMB', 
     'Persistent Mutations', 'Clonal Persistent Mutations', 'Clonal Mutations']]
dfm1.columns = ['Noushin_stage', 'Noushin_OST', 'Noushin_OSS', 'Noushin_TMB', 'Noushin_pTMB', 'Noushin_cpTMB', 'Noushin_cTMB']
dfm1[dfm1.columns[1:]] = np.log2(dfm1[dfm1.columns[1:]] + 1)
dfm1.Noushin_OSS = dfm1.Noushin_OSS.map({1:'Dead', 0:'Alive'})

tcga_patient1 = tcga_patient.join(dfm1)
tcga_patient1['os_time'] = np.log2(tcga_patient1.os_time+1)
tcga_patient1['pfi_time'] = np.log2(tcga_patient1.pfi_time+1)

/tmp/ipykernel_46635/3931213934.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm1[dfm1.columns[1:]] = np.log2(dfm1[dfm1.columns[1:]] + 1)
/tmp/ipykernel_46635/3931213934.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm1.Noushin_OSS = dfm1.Noushin_OSS.map({1:'Dead', 0:'Alive'})


In [60]:
tcga_patient1.to_pickle(os.path.join(tcga_data_path, 'PATIENTS.v2.TABLE'))

In [17]:
#np.log(dfm1.Noushin_pTMB+1).hist(bins=20)